In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DateType
import pyspark.sql.functions as f

In [2]:
#SPARK CONFIG
warehouse_location = 'hdfs://hdfs-nn:9000/demo/'

builder = SparkSession \
    .builder \
    .appName("Python Spark") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c6678bd5-f931-40e4-b1ef-729cb587a45d;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 654ms :: artifacts dl 29ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-run

In [3]:
spark.sql(
    """
    DROP TABLE IF EXISTS gold_leagues
    """
)

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

DataFrame[]

In [4]:
#DROP TABLE
spark.sql(
    """
    DROP TABLE IF EXISTS gold_leagues.stats
    """
)

DataFrame[]

In [5]:
# create gold database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS gold_leagues LOCATION 'hdfs://hdfs-nn:9000/demo/gold/projeto/gold_leagues.db/'
    """
)


spark.sql(
    """
    CREATE EXTERNAL TABLE gold_leagues.stats (
    
        Div STRING,
        Team STRING,
        Date STRING,
        Media_golos DOUBLE,
        Media_remates DOUBLE,
        Media_remates_baliza DOUBLE,
        Media_cantos DOUBLE,
        Eficacia DOUBLE
        
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/demo/gold/projeto/gold_leagues.db/stats/'
    """
)

22/01/17 13:51:13 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `gold_leagues`.`stats` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
22/01/17 13:51:13 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [6]:
from pyspark.sql.functions import substring, avg, sum, round
from pyspark.sql.functions import col, concat_ws, concat, lit, desc, asc

# read data from tables in projeto.db
tabela = spark.read.format("delta").load("hdfs://hdfs-nn:9000/demo/silver/F_SP_League.db/deltalake_table/")


silver_table = tabela \
    .withColumn("Date", concat(lit("20"), substring(col("Date"), 7, 8)))


silver_table.show()

+--------------------+--------------------+----+----+----+----+---+---+---+---+---+---+------+
|            HomeTeam|            AwayTeam| FTR|Date|FTHG|FTAG| HS| AS|HST|AST| HC| AC|   Div|
+--------------------+--------------------+----+----+----+----+---+---+---+---+---+---+------+
|               Reims|            Paris SG|Draw|2014|   2|   2|  9| 16|  3|  6|  1|  5|France|
|              Bastia|           Marseille|Draw|2014|   3|   3| 13|  9|  4|  4|  1|  4|France|
|Evian Thonon Gail...|                Caen|Away|2014|   0|   3| 10| 12|  2|  7|  5|  6|France|
|            Guingamp|          St Etienne|Away|2014|   0|   2|  6|  7|  3|  2|  4|  5|France|
|               Lille|                Metz|Draw|2014|   0|   0| 14|  2|  3|  1|  8|  3|France|
|         Montpellier|            Bordeaux|Away|2014|   0|   1| 15|  7|  3|  3|  5|  1|France|
|              Nantes|                Lens|Home|2014|   1|   0| 14|  5|  4|  2|  3|  2|France|
|                Nice|            Toulouse|Home|20

In [7]:
homeTeam_table = silver_table \
    .groupBy("HomeTeam", "Date", "Div") \
    .agg(
        avg(silver_table.FTHG).alias("FTHG"),
        avg(silver_table.HS).alias("HSH"),
        avg(silver_table.HST).alias("HSTH"),
        avg(silver_table.HC).alias("HCH"),
        sum(silver_table.HS).alias("SHS"),
        sum(silver_table.FTHG).alias("SFTHG")
    ) \
    .withColumnRenamed("HomeTeam", "Team") 

homeTeam_table.show(n=30)
homeTeam_table.count()


+--------------------+----+------+------------------+------------------+------------------+------------------+---+-----+
|                Team|Date|   Div|              FTHG|               HSH|              HSTH|               HCH|SHS|SFTHG|
+--------------------+----+------+------------------+------------------+------------------+------------------+---+-----+
|            Bordeaux|2019|France|               1.0|11.222222222222221|               4.0| 5.555555555555555|101|    9|
|               Lille|2016|France|              1.25|             13.45|               4.4|               5.4|269|   25|
|             Sevilla|2014| Spain|               2.0|            13.625|             5.625|              7.25|109|   16|
|         Real Madrid|2016| Spain|3.3333333333333335| 19.38888888888889| 7.722222222222222| 6.611111111111111|349|   60|
|             Leganes|2017| Spain|1.1666666666666667|12.444444444444445| 4.222222222222222| 4.722222222222222|224|   21|
|               Betis|2018| Spai

263

In [8]:
awayTeam_table = silver_table \
    .groupBy("AwayTeam", "Date", "Div") \
    .agg(
        avg(silver_table.FTAG).alias("FTAG"),
        avg(silver_table.AS).alias("ASA"),
        avg(silver_table.AST).alias("ASTA"),
        avg(silver_table.AC).alias("ACA"),
        sum(silver_table.AS).alias("SAS"),
        sum(silver_table.FTAG).alias("SFTAG")
    ) \
    .withColumnRenamed("AwayTeam", "Team") 

awayTeam_table.show(n=30)
awayTeam_table.count()


+--------------------+----+------+-------------------+------------------+------------------+------------------+---+-----+
|                Team|Date|   Div|               FTAG|               ASA|              ASTA|               ACA|SAS|SFTAG|
+--------------------+----+------+-------------------+------------------+------------------+------------------+---+-----+
|            Bordeaux|2019|France| 0.4166666666666667| 8.833333333333334|              2.25|               4.0|106|    5|
|               Lille|2016|France| 0.9444444444444444| 10.61111111111111| 3.888888888888889| 3.388888888888889|191|   17|
|             Sevilla|2014| Spain| 1.2857142857142858| 8.428571428571429| 4.142857142857143| 4.571428571428571| 59|    9|
|         Real Madrid|2016| Spain|                2.5| 16.77777777777778| 6.722222222222222| 6.111111111111111|302|   45|
|             Leganes|2017| Spain|                0.7|             10.05|               3.2|               3.8|201|   14|
|               Betis|20

263

In [9]:
#join of the two dataFrames
join_table = homeTeam_table.join(awayTeam_table,['Team', 'Date', 'Div'], how='left') \
    .withColumnRenamed("Team", "Equipa") \
    .withColumnRenamed("Div", "Liga") \
    .withColumnRenamed("Date", "Data") \
    .orderBy("Date", "Liga") \
    .withColumn("Media_golos", round(((col("FTAG") + col("FTHG"))/2), 2)) \
    .withColumn("Media_remates", round(((col("HSH") + col("ASA"))/2), 2)) \
    .withColumn("Media_remates_baliza", round(((col("HSTH") + col("ASTA"))/2), 2)) \
    .withColumn("Media_cantos", round(((col("HCH") + col("ACA"))/2), 2)) \
    .withColumn("Eficacia", round(((col("SFTHG") + col("SFTAG")) / (col("SHS") + col("SAS")) * 100), 2)) \
    .select("Liga", "Equipa", "Data", "Media_golos", "Media_remates", "Media_remates_baliza", "Media_cantos", "Eficacia")


#WRITE TO DELTA TABLE
join_table \
    .write \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/demo/gold/projeto/gold_leagues.db/stats/")



join_table.show()
join_table.count()


22/01/17 13:51:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


+------+--------------------+----+-----------+-------------+--------------------+------------+--------+
|  Liga|              Equipa|Data|Media_golos|Media_remates|Media_remates_baliza|Media_cantos|Eficacia|
+------+--------------------+----+-----------+-------------+--------------------+------------+--------+
|France|                Metz|2014|       0.93|        11.52|                3.82|        4.07|    7.83|
|France|            Bordeaux|2014|       1.39|        11.79|                3.55|        3.99|   11.71|
|France|              Monaco|2014|       1.21|        12.11|                4.74|        5.64|   10.04|
|France|                Lens|2014|       1.06|         10.6|                3.29|        3.76|    10.0|
|France|           Marseille|2014|       1.98|        15.66|                5.77|        6.44|   12.71|
|France|                Lyon|2014|       2.08|        14.91|                5.96|        4.76|   14.08|
|France|              Nantes|2014|        1.0|        11.77|    

263

In [10]:
# check the results in the table
spark.table("gold_leagues.stats").toPandas()

,Liga,Equipa,Data,Media_golos,Media_remates,Media_remates_baliza,Media_cantos,Eficacia
0,France,Lille,2016,1.10,12.03,4.14,4.39,9.13
1,France,Bastia,2016,0.87,10.42,3.50,4.16,8.33
2,France,Bordeaux,2016,1.24,11.13,3.84,4.50,11.11
3,France,Caen,2016,1.04,12.33,4.18,4.29,8.35
4,France,Monaco,2016,2.29,11.84,5.07,5.07,19.38
...,...,...,...,...,...,...,...,...
258,Spain,Levante,2014,0.75,10.94,2.63,4.19,6.86
259,Spain,Espanol,2014,1.25,9.38,3.56,5.31,13.33
260,Spain,Elche,2014,0.81,10.88,3.50,4.50,7.47
261,Spain,La Coruna,2014,0.81,8.69,2.94,3.81,9.35


In [11]:
spark.sql(     """     GENERATE symlink_format_manifest FOR TABLE delta. `hdfs://hdfs-nn:9000/demo/gold/projeto/gold_leagues.db/stats/`     """ ).show()

++
||
++
++



In [12]:
spark.sql("""
DROP TABLE IF EXISTS gold_leagues.stats_presto
""").show()

spark.sql(
    """
    CREATE EXTERNAL TABLE gold_leagues.stats_presto (

        Liga STRING,
        Equipa STRING,
        Data STRING,
        Media_golos DOUBLE,
        Media_remates DOUBLE,
        Media_remates_baliza DOUBLE,
        Media_cantos DOUBLE,
        Eficacia DOUBLE
        )


ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/demo/gold/projeto/gold_leagues.db/stats/_symlink_format_manifest/'
""").show()

22/01/17 13:51:50 WARN DataSource: All paths were ignored:
  hdfs://hdfs-nn:9000/demo/gold/projeto/gold_leagues.db/stats/_symlink_format_manifest


++
||
++
++

++
||
++
++

